In [1]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import time
import findspark
findspark.init()

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession

from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='localhost:9092')


#consumer key, consumer secret, access token, access secret.
ckey="zQF1YoRRmPZhuKw6LWz4VyoY1"
csecret="Lsf7dwpzfmLMn6s9bEpArmDefar2UFrjRAhpxstH7ijpUfJeK2"
atoken="1017435190556323840-qK4GWsSupuIOYSw6gSiTd64oINtc6a"
asecret="MFe6P1Iq1r15HfN3PKtUCntfHb0pQULlYrsQXddfTx8y2"

my_spark = SparkSession.builder.appName("twitterData").master("local").getOrCreate() 
sc = my_spark.sparkContext
    
class listener(StreamListener):

    def on_data(self, data):
        try:
            all_data = json.loads(data)
            
            tweet = all_data["text"].lower()
            print(tweet)
            future = producer.send('tweets', bytes(tweet, encoding= 'utf-8'))
            producer.flush()
                                          
            return True
        except:
            return True

    def on_error(self, status):
        print(status)

movies = my_spark.read.format("com.mongodb.spark.sql.DefaultSource") \
          .option("uri","mongodb://localhost:27017/movieDatabase.movies").load()
hashtag_list = []
for h in movies.select('title_hashtag').toPandas()['title_hashtag']: # get all documents
    hashtag_list.append(h)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
filterWords = hashtag_list
twitterStream.filter(track=filterWords)


rt @nctsmtown: nct 2018 총출동 할로윈 전야제┃happy halloween eve 🎃👻🍭

https://t.co/ht653rtzbv

#ch_nct #채널nct 
#nct #nct127 #nctdream
#nct2018 #hall…
rt @marklee_th: [official] 181121 nctsmtown's twitter update
แฮชาน : พี่มาร์คเคยดูโคนันมั้ยครับ
มาร์ค : ชื่อของผมคือ..

#nct #nct127 #nctdr…
rt @nctsmtown: nct 2018 총출동 할로윈 전야제┃happy halloween eve 🎃👻🍭

https://t.co/ht653rtzbv

#ch_nct #채널nct 
#nct #nct127 #nctdream
#nct2018 #hall…
rt @mamakonn: 描きました。 #俺たちはヴェノムだ #venom https://t.co/mb2slqfus1
rt @li_ili_ili_il: ヴェノムかわいいは周知されたと思うのでエディもかわいいよ！を主張してみる画像　エディもかわいいよ　#venom https://t.co/saqlcacfkp
ふふっ
出たでた
ハッシュタグでフレディの絵文字❤
#bohemianrhapsody 
応援上映見に行きたいよおおおお
でも、尽く仕事に稽古…😭
フレディの絵文字？ #bohemianrhapsody
rt @cafemonochrome: #ヴェノム 公開記念
"#venom inspiredcafe" 
映画コンセプトカフェ@cafemonochrome 
12/2(日)迄会期延長！
明後日11/23(金)数量限定
オフィシャルグッズ入荷致します。
ノベルティ情報、グッズ…
rt @nctsmtown: nct 2018 총출동 할로윈 전야제┃happy halloween eve 🎃👻🍭

https://t.co/ht653rtzbv

#ch_nct #채널nct 
#nct #nct127 #nctdream
#nct2018 #hall…
rt @nctsmtown: nct 2018 총출동 할로윈

how am i supposed to wait now with that tease 😩
these kids are so funny 😂
rt @nctsmtown: nct 2018 총출동 할로윈 전야제┃happy halloween eve 🎃👻🍭

https://t.co/ht653rtzbv

#ch_nct #채널nct 
#nct #nct127 #nctdream
#nct2018 #hall…
rt @nctsmtown: nct 2018 총출동 할로윈 전야제┃happy halloween eve 🎃👻🍭

https://t.co/ht653rtzbv

#ch_nct #채널nct 
#nct #nct127 #nctdream
#nct2018 #hall…
rt @queen40jp: フレディの命日に、ニコ生で映画公開&amp;大ヒット記念24時間特番が決定！

これが伝説のバンドqueenだ!!　映画「ボヘミアン・ラプソディ」大ヒット記念24時間特番
11月24日(土) 0:00～11月25日(日) 0:00
https://t.co…
rt @nctsmtown: nct 2018 총출동 할로윈 전야제┃happy halloween eve 🎃👻🍭

https://t.co/ht653rtzbv

#ch_nct #채널nct 
#nct #nct127 #nctdream
#nct2018 #hall…
rt @gingerann6: yummy #treats!!! #rt #halloween #milfmafia #titsout https://t.co/nini6ayylv
rt @nctsmtown: nct 2018 총출동 할로윈 전야제┃happy halloween eve 🎃👻🍭

https://t.co/ht653rtzbv

#ch_nct #채널nct 
#nct #nct127 #nctdream
#nct2018 #hall…
rt @nctsmtown: nct 2018 총출동 할로윈 전야제┃happy halloween eve 🎃👻🍭

https://t.co/ht653rtzbv

#ch_nct #채널nct 
#nct #nct127 #nctdream


rt @marklee_th: [official] 181121 nctsmtown's twitter update
nct 2018 สุขสันต์วันฮาโลวีนอีฟ 🎃👻🍭

🔗 https://t.co/4oeoywx7gh 
#ch_nct #채널nct…
rt @nctsmtown: nct 2018 총출동 할로윈 전야제┃happy halloween eve 🎃👻🍭

https://t.co/ht653rtzbv

#ch_nct #채널nct 
#nct #nct127 #nctdream
#nct2018 #hall…
rt @queen40jp: フレディの命日に、ニコ生で映画公開&amp;大ヒット記念24時間特番が決定！

これが伝説のバンドqueenだ!!　映画「ボヘミアン・ラプソディ」大ヒット記念24時間特番
11月24日(土) 0:00～11月25日(日) 0:00
https://t.co…
rt @marklee_th: [official] 181121 nctsmtown's twitter update
แฮชาน : พี่มาร์คเคยดูโคนันมั้ยครับ
มาร์ค : ชื่อของผมคือ..

#nct #nct127 #nctdr…
rt @nctsmtown: nct 2018 총출동 할로윈 전야제┃happy halloween eve 🎃👻🍭

https://t.co/ht653rtzbv

#ch_nct #채널nct 
#nct #nct127 #nctdream
#nct2018 #hall…
rt @nctsmtown: nct 2018 총출동 할로윈 전야제┃happy halloween eve 🎃👻🍭

https://t.co/ht653rtzbv

#ch_nct #채널nct 
#nct #nct127 #nctdream
#nct2018 #hall…
rt @nctsmtown: nct 2018 총출동 할로윈 전야제┃happy halloween eve 🎃👻🍭

https://t.co/ht653rtzbv

#ch_nct #채널nct 
#nct #nct127 #nctdream
#nct2018 #hall…
rt @hub__d

KeyboardInterrupt: 